<center>
<img src="https://i.imgur.com/DW7NRwf.png" title="ElementalsAI" width=80%/>
</center>

# Introduction

Deep Learning has been very successful in solving many complex problems in the real world. However, solving most of these challenges requires data. Additionally, having data is not just enough. The data has to be cleansed, annotated, and organized. Moreover, data in many domains is not accessible due to privacy constraints (Especially in the medical domain). Hence it becomes pertinent to create models that can learn from a limited amount of data.

<br>

## Problem Statement:
In this task, you are given a small dataset of the microscopic view of the cells.

*Your goal is to build a model that accurately predicts the cell regions as shown in the **Label Image**.*

#### **Input Image**

<a href="https://imgur.com/aLDNHwu"><img src="https://i.imgur.com/aLDNHwu.png" title="source: imgur.com" /></a>

#### **Label Image**

<a href="https://imgur.com/s1mIkFE"><img src="https://i.imgur.com/s1mIkFE.png" title="source: imgur.com" /></a>


## Dataset:

The dataset contains 30 training images along with the labels.

[**Dataset Link**](https://drive.google.com/drive/folders/1678Tggykj46SpJZS9mKMKHw7YFmiGMc8?usp=sharing)


## Evaluation

Share the submission Jupyter/Colab notebook with the relevant explanation. code and models. We will evaluate your model on our **test dataset**.

<br><br>
****Note that this is an interview assignment & is not involved in the development of any software or solution.**

### Instructions

1. Make a copy of this notebook to start editing & add your solution.
2. A dataset folder has been shared with you to train and test.
2. You can use any framework to develop the solution (Pytorch, Keras, Tensorflow, Theano, Caffe etc.).
3. This assignment is a great medium to get to know you better. Please feel free to connect, interact & develop the solution. I would be more than willing to help you out in any issues or problems that you face while solving the challenge. You can connect with me at suraj.donthi@elementals.ai.
4. The goal of this task is to understand how you approach solving a problem. The more you connect while developing the solution the better I will be able to understand you.
5. Submission Files:
    - Colab Notebook Link with Solutions Approach and Code Solution.
    - Trained Model Link.
    - Any other necessary files.
5. **Submission deadline: Within 5 days of recieving the assignment, no later than 12 AM IST on the due date. The exact due date shall be mentioned in the email or the portal where you recieve the assignment.** (You can share the Colab Notebook to the gmail address surajdonthi.th@gmail.com)

## Explain the technique you will use to solve the problem in detail.

- Include any model architectures, equations, diagrams etc. that is required to explain how you are going to solve the problem.

To build a image segmetation model u need a relatively good dataset but the provided dataset contains only 30 images so i uses image segmentation to generate 100 images for training
This model uses U-net architecture
The U-Net model is a powerfull for image segmentation tasks. The model is relatively easy to train and can achieve state-of-the-art results on a variety of image segmentation datasets.
This model is solely build by me so it has many problems , I have to fine tune it
for now i trained the model
This model contain 4 main scripts
Fist one is for augmentation
Second for preprocessing the images
Third for converting the images and masks into .npy file
Fourth is the training script itself```





```
================== `Your answer here. (Double click to edit)` ==================

## Implement the solution below.

#AUGMENTATION The given dataset is too small to train a model for cell segmentaion thats why i use this script to generate 2000 images and masks with the given data


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from skimage.transform import AffineTransform, warp
from skimage import io, img_as_ubyte
import random
import os
from scipy.ndimage import rotate

import albumentations as A
images_to_generate=1000




images_path="/content/gdrive/MyDrive/Projects/train_data/images" #path to original images
masks_path = "/content/gdrive/MyDrive/Projects/train_data/masks"
img_augmented_path="/content/gdrive/MyDrive/Projects/aug/images/" # path to store aumented images
msk_augmented_path="/content/gdrive/MyDrive/Projects/aug/masks" # path to store aumented images
images=[] # to store paths of images from folder
masks=[]

for im in os.listdir(images_path):  # read image name from folder and append its path into "images" array
    images.append(os.path.join(images_path,im))

for msk in os.listdir(masks_path):  # read image name from folder and append its path into "images" array
    masks.append(os.path.join(masks_path,msk))


aug = A.Compose([
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.HorizontalFlip(p=1),
    A.Transpose(p=1),
    #A.ElasticTransform(p=1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
    A.GridDistortion(p=1)
    ]
)

#random.seed(42)

i=1   # variable to iterate till images_to_generate


while i<=images_to_generate:
    number = random.randint(0, len(images)-1)  #PIck a number to select an image & mask
    image = images[number]
    mask = masks[number]
    print(image, mask)
    #image=random.choice(images) #Randomly select an image name
    original_image = io.imread(image)
    original_mask = io.imread(mask)

    augmented = aug(image=original_image, mask=original_mask)
    transformed_image = augmented['image']
    transformed_mask = augmented['mask']


    new_image_path= "%s/augmented_image_%s.tif" %(img_augmented_path, i)
    new_mask_path = "%s/augmented_mask_%s.tif" %(msk_augmented_path, i)
    io.imsave(new_image_path, transformed_image)
    io.imsave(new_mask_path, transformed_mask)
    i =i+1

In [1]:
import os
import numpy as np
from PIL import Image
import tifffile
import glob

def load_image(path):
  """Load an image from a file.

  Args:
    path: The path to the image file.

  Returns:
    A PIL Image object.
  """

  image = tifffile.imread(path)
  image = Image.fromarray(image)

  return image

def load_mask(path):
  """Load a mask from a file.

  Args:
    path: The path to the mask file.

  Returns:
    A PIL Image object.
  """

  mask = tifffile.imread(path)
  mask = Image.fromarray(mask)

  return mask

def preprocess_image(image):
  """Preprocess an image for U-Net cell region prediction.

  Args:
    image: A PIL Image object.

  Returns:
    A preprocessed image tensor.
  """

  # Resize the image to the desired size.
  image = image.resize((256, 256))

  # Normalize the pixel values to [0, 1].
  image = np.asarray(image, dtype=np.float32) / 255.0

  # Add a channel dimension.
  image = image[..., np.newaxis]

  return image

def preprocess_mask(mask):
  """Preprocess a mask for U-Net cell region prediction.

  Args:
    mask: A PIL Image object.

  Returns:
    A preprocessed mask tensor.
  """

  # Resize the mask to the desired size.
  mask = mask.resize((256, 256))

  # Convert the mask to a binary mask.
  mask = np.asarray(mask, dtype=np.uint8)
  mask = mask[..., 0]
  mask = np.where(mask > 0, 1, 0)

  # Add a channel dimension.
  mask = mask[..., np.newaxis]

  return mask

def preprocess_dataset(image_paths, mask_paths):
  """Preprocess a dataset of images and masks for U-Net cell region prediction.

  Args:
    image_paths: A list of paths to image files.
    mask_paths: A list of paths to mask files.

  Returns:
    A tuple of preprocessed image and mask tensors.
  """

  preprocessed_images = []
  preprocessed_masks = []

  for image_path, mask_path in zip(image_paths, mask_paths):
    image = load_image(image_path)
    mask = load_mask(mask_path)

    preprocessed_image = preprocess_image(image)
    preprocessed_mask = preprocess_mask(mask)

    preprocessed_images.append(preprocessed_image)
    preprocessed_masks.append(preprocessed_mask)

  return preprocessed_images, preprocessed_masks

def main():
  # Get the paths to the image and mask directories.
  image_dir = '/content/gdrive/MyDrive/Projects/aug/images'
  mask_dir = '/content/gdrive/MyDrive/Projects/aug/masks'

  # Get the paths to all of the images and masks.
  image_paths = glob.glob(os.path.join(image_dir, '*.tif'))
  mask_paths = glob.glob(os.path.join(mask_dir, '*.tif'))

  # Preprocess the images and masks.
  preprocessed_images, preprocessed_masks = preprocess_dataset(image_paths, mask_paths)

  # Train or predict the U-Net model using the preprocessed images and masks
  # ...

if __name__ == '__main__':
  main()

In [2]:
import os
import numpy as np
from PIL import Image

def convert_to_npy(images, masks, output_dir):
  """Converts images and masks to NumPy arrays and saves them to a directory.

  Args:
    images: A list of paths to image files.
    masks: A list of paths to mask files.
    output_dir: The directory to save the NumPy arrays to.
  """

  # Create the output directory if it does not exist.
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  # Convert the images and masks to NumPy arrays.
  image_arrays = []
  mask_arrays = []
  for image_path, mask_path in zip(images, masks):
    image = Image.open(image_path)
    mask = Image.open(mask_path)

    image_array = np.array(image)
    mask_array = np.array(mask)

    image_arrays.append(image_array)
    mask_arrays.append(mask_array)

  # Save the NumPy arrays to the output directory.
  np.save(os.path.join(output_dir, 'images.npy'), image_arrays)
  np.save(os.path.join(output_dir, 'masks.npy'), mask_arrays)

if __name__ == '__main__':
  # Get the paths to the image and mask directories.
  image_dir = '/content/gdrive/MyDrive/Projects/aug/images'
  mask_dir = '/content/gdrive/MyDrive/Projects/aug/masks'

  # Get the paths to all of the images and masks.
  image_paths = glob.glob(os.path.join(image_dir, '*.tif'))
  mask_paths = glob.glob(os.path.join(mask_dir, '*.tif'))

  # Convert the images and masks to NumPy arrays and save them to a directory.
  output_dir = '/content/gdrive/MyDrive/Projects/aug/npy'
  convert_to_npy(image_paths, mask_paths, output_dir)

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate

class UNet(tf.keras.Model):
    def __init__(self):
        super(UNet, self).__init__()

        # Encoder
        self.encoder = tf.keras.Sequential([
            # Convert the input to a floating-point data type
            tf.keras.layers.Lambda(lambda x: tf.cast(x, tf.float32)),

            Conv2D(32, (3, 3), activation='relu', padding='same'),
            MaxPooling2D((2, 2)),

            Conv2D(64, (3, 3), activation='relu', padding='same'),
            MaxPooling2D((2, 2)),

            Conv2D(128, (3, 3), activation='relu', padding='same'),
            MaxPooling2D((2, 2)),

            Conv2D(256, (3, 3), activation='relu', padding='same'),
            MaxPooling2D((2, 2)),
        ])

        # Decoder
        self.decoder = tf.keras.Sequential([
            UpSampling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu', padding='same'),

            UpSampling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu', padding='same'),

            UpSampling2D((2, 2)),
            Conv2D(32, (3, 3), activation='relu', padding='same'),

            UpSampling2D((2, 2)),
            Conv2D(1, (1, 1), activation='sigmoid', padding='same'),
        ])

    def call(self, inputs):
        # Encode the input image
        encoder_output = self.encoder(inputs)

        # Decode the encoded features
        decoder_output = self.decoder(encoder_output)

        return decoder_output

# Training script

def train(model, train_images_path, train_masks_path, epochs=100):
  """Trains the U-Net model.

  Args:
    model: The U-Net model.
    train_images_path: The path to the training images .npy file.
    train_masks_path: The path to the training masks .npy file.
    epochs: The number of epochs to train the model for.
  """

  # Load the training images and masks
  train_images = np.load(train_images_path)
  train_masks = np.load(train_masks_path)

  # Reshape the input images
  train_images = train_images.reshape((-1, 512, 512, 1))
  train_masks = train_masks.reshape((-1, 512, 512, 1))

  # Build the model
  model.call(train_images[:1])

  # Compile the model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  # Train the model
  model.fit(train_images, train_masks, epochs=epochs)

  # Save the model weights
  model.save_weights('unet_model_weights.h5')

# Example usage

# Get the paths to the training images and masks .npy files
train_images_path = '/content/gdrive/MyDrive/Projects/aug/npy/images.npy'
train_masks_path = '/content/gdrive/MyDrive/Projects/aug/npy/masks.npy'

# Create the U-Net model
model = UNet()

# Train the model
train(model, train_images_path, train_masks_path)

# Save the model weights
model.save_weights('unet_model_weights.h5')

Epoch 1/100
32/32 [==============================] - 17s 427ms/step - loss: -4051240960.0000 - accuracy: 0.0136
Epoch 2/100
32/32 [==============================] - 14s 432ms/step - loss: -13058077360128.0000 - accuracy: 0.0000e+00
Epoch 3/100
32/32 [==============================] - 14s 435ms/step - loss: -2105272440979456.0000 - accuracy: 0.0000e+00
Epoch 4/100
32/32 [==============================] - 14s 437ms/step - loss: -79876195453239296.0000 - accuracy: 0.0000e+00
Epoch 5/100
32/32 [==============================] - 14s 439ms/step - loss: -1277583995525136384.0000 - accuracy: 0.0000e+00
Epoch 6/100
32/32 [==============================] - 14s 441ms/step - loss: -11697305085187981312.0000 - accuracy: 0.0000e+00
Epoch 7/100
32/32 [==============================] - 14s 433ms/step - loss: nan - accuracy: 0.0726
Epoch 8/100
32/32 [==============================] - 13s 414ms/step - loss: nan - accuracy: 0.2195
Epoch 9/100
32/32 [==============================] - 13s 414ms/step - loss